In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# 📦 Step 1: Install Dependencies

!pip install opencv-python-headless tensorflow keras --quiet

In [ ]:
# 📁 Step 2: Extract Frames from .avi Videos
import cv2
import os

# This function extracts frames from .avi files inside class folders
# and saves them in a corresponding output folder structure

def extract_frames_from_videos(video_dir, output_dir, frame_rate=10):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    for class_folder in os.listdir(video_dir):
        class_path = os.path.join(video_dir, class_folder)
        output_class_path = os.path.join(output_dir, class_folder)
        os.makedirs(output_class_path, exist_ok=True)

        for video_name in os.listdir(class_path):
            if not video_name.endswith(".avi"):
                continue

            video_path = os.path.join(class_path, video_name)
            cap = cv2.VideoCapture(video_path)

            count = 0
            saved = 0
            while cap.isOpened():
                ret, frame = cap.read()
                if not ret:
                    break
                if count % frame_rate == 0:
                    frame_file = f"{os.path.splitext(video_name)[0]}_{saved}.jpg"
                    frame_path = os.path.join(output_class_path, frame_file)
                    cv2.imwrite(frame_path, frame)
                    saved += 1
                count += 1

            cap.release()

# Extract frames from AVI videos in Train and Test directories
extract_frames_from_videos("/kaggle/input/smartcity-cctv-violence-detection-dataset-scvd/SCVD/SCVD_converted/Train", "/kaggle/working/frames/train")
extract_frames_from_videos("/kaggle/input/smartcity-cctv-violence-detection-dataset-scvd/SCVD/SCVD_converted/Test", "/kaggle/working/frames/test")


In [ ]:
# 📊 Step 3: Load Image Data for Training
from tensorflow.keras.preprocessing.image import ImageDataGenerator

img_height, img_width = 224, 224
batch_size = 32

train_dir = "/kaggle/working/frames/train"
test_dir = "/kaggle/working/frames/test"

datagen = ImageDataGenerator(rescale=1./255)

train_generator = datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'
)

test_generator = datagen.flow_from_directory(
    test_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'
)


In [ ]:
# 🧠 Step 4: Create a Simple CNN Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(img_height, img_width, 3)),
    MaxPooling2D(2,2),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(3, activation='softmax')  # 3 classes: Normal, Violence, Weaponized
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()


In [ ]:

# 💾 Step 5: Define Callbacks (Checkpoint + Learning Rate Scheduler)
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau

checkpoint_cb = ModelCheckpoint(
    "/kaggle/working/best_model.keras",
    save_best_only=True,
    monitor='val_accuracy',
    verbose=1
)

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, verbose=1)

callbacks = [checkpoint_cb, reduce_lr]

In [ ]:
# 🚀 Step 6: Train the Model
EPOCHS = 10

history = model.fit(
    train_generator,
    epochs=EPOCHS,
    validation_data=test_generator,
    callbacks=callbacks
)


In [ ]:
# Save the full model (architecture + weights)
model.save("/kaggle/working/final_violence_model1.h5")


In [ ]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array
import os

# Load the trained model
model = load_model("/kaggle/working/final_violence_model.keras")  # adjust path if needed

# Class labels (adjust if your labels are different)
class_names = ['Normal', 'Violence', 'Weapon']  # or ['NonViolent', 'Violent'] if using 2-class

# Function to predict on a video
def predict_video(video_path, img_height=224, img_width=224, frame_skip=10):
    cap = cv2.VideoCapture(video_path)
    predictions = []

    frame_count = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        if frame_count % frame_skip == 0:
            resized_frame = cv2.resize(frame, (img_width, img_height))
            img = img_to_array(resized_frame) / 255.0
            img = np.expand_dims(img, axis=0)

            pred = model.predict(img)
            predicted_class = np.argmax(pred)
            predictions.append(predicted_class)

        frame_count += 1
    
    cap.release()

    # Majority voting
    if not predictions:
        return "No frames were processed."
    
    final_class = max(set(predictions), key=predictions.count)
    return f"Prediction: {class_names[final_class]}"

# 🔍 Test with your uploaded video
test_video_path = "/kaggle/input/your-test-folder/sample.avi"  # 👈 Update this path

result = predict_video(test_video_path)
print(result)


In [25]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array
import os


In [26]:
model = load_model('/kaggle/input/violence-detection/tensorflow2/default/1/final_violence_model1.h5')  # Replace with your actual path


In [27]:
IMG_SIZE = 224  # Adjust if your model uses different size

def preprocess_frame(frame):
    frame = cv2.resize(frame, (IMG_SIZE, IMG_SIZE))
    frame = frame.astype("float32") / 255.0
    frame = img_to_array(frame)
    frame = np.expand_dims(frame, axis=0)
    return frame


In [28]:
def predict_video(video_path, frame_skip=30):
    cap = cv2.VideoCapture(video_path)
    predictions = []

    frame_count = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        if frame_count % frame_skip == 0:
            processed = preprocess_frame(frame)
            pred = model.predict(processed)[0]
            predictions.append(pred)
        frame_count += 1
    cap.release()

    # Average prediction across frames
    if len(predictions) == 0:
        return "No frames processed"

    avg_prediction = np.mean(predictions, axis=0)
    class_idx = np.argmax(avg_prediction)
    labels = ["Normal", "Violence", "Weapon"]  # Modify as per your model’s output

    return labels[class_idx]


In [36]:
# video_path = "/kaggle/input/smartcity-cctv-violence-detection-dataset-scvd/SCVD/SCVD_converted_sec_split/Test/Weaponized/Weaponized-Test030.avi"  
# video_path = "/kaggle/input/smartcity-cctv-violence-detection-dataset-scvd/SCVD/SCVD_converted_sec_split/Test/Normal/Normal-Test002.avi" 
video_path = "/kaggle/input/smartcity-cctv-violence-detection-dataset-scvd/SCVD/SCVD_converted_sec_split/Test/Violence/Violence-Test009.avi" # Make sure you uploaded this
result = predict_video(video_path)
print("🎥 Prediction for the video:", result)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
🎥 Prediction for the video: Violence


In [ ]:
import cv2

def convert_to_avi(input_path, output_path):
    cap = cv2.VideoCapture(input_path)
    if not cap.isOpened():
        print("❌ Error opening video file.")
        return

    width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))  
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)) 
    fps    = cap.get(cv2.CAP_PROP_FPS)

    fourcc = cv2.VideoWriter_fourcc(*'XVID')  # Use XVID codec for AVI
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        out.write(frame)

    cap.release()
    out.release()
    print(f"✅ Converted video saved as: {output_path}")


In [ ]:
input_video = "/kaggle/input/your-video-folder/your_video.mp4"  # your original video
output_video = "/kaggle/input/test-set/Villupuram School and college boy _Fight  _college _fight _trending _status _shorts _chennai(360P).mp4"             # avi output

convert_to_avi(input_video, output_video)


In [ ]:
result = predict_video(output_video)
print("🎯 Prediction:", result)


In [ ]:
import tensorflow as tf
print(tf.__version__)


In [ ]:
import keras
print(keras.__version__)


In [ ]:
import cv2
import numpy as np
from keras.models import load_model
from keras.preprocessing.image import img_to_array

# === CONFIG ===
IMG_SIZE = 224  # Adjust based on your model
LABELS = ["Normal", "Violence", "Weapon"]  # Adjust according to your model output
FRAME_SKIP = 30  # Process every 30th frame

# === Load Model ===
model_path = "/kaggle/input/violence-detection/tensorflow2/default/1/final_violence_model1.h5"  # <- Update this
model = load_model(model_path, compile=False)

# === Frame Preprocessing ===
def preprocess_frame(frame):
    frame = cv2.resize(frame, (IMG_SIZE, IMG_SIZE))
    frame = frame.astype("float32") / 255.0
    frame = img_to_array(frame)
    frame = np.expand_dims(frame, axis=0)
    return frame

# === Predict + Save New Video ===
def predict_and_save_video(input_video_path, output_video_path):
    cap = cv2.VideoCapture(input_video_path)

    if not cap.isOpened():
        print("❌ Error: Cannot open input video.")
        return

    # Get video properties
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)

    # Define video writer
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

    frame_count = 0
    current_label = "Detecting..."

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        if frame_count % FRAME_SKIP == 0:
            processed = preprocess_frame(frame)
            pred = model.predict(processed)[0]
            label_idx = np.argmax(pred)
            current_label = LABELS[label_idx]

        # Overlay prediction on frame
        cv2.putText(frame, f"Prediction: {current_label}", (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

        out.write(frame)
        frame_count += 1

    cap.release()
    out.release()
    print("✅ Prediction video saved:", output_video_path)

# === File Paths ===
input_video_path = "/kaggle/input/test-set/Villupuram School and college boy _Fight  _college _fight _trending _status _shorts _chennai(360P).mp4"
output_video_path = "predicted_output.avi"

predict_and_save_video(input_video_path, output_video_path)


In [ ]:
from IPython.display import Video
Video("predicted_output.avi", embed=True)


In [43]:
import os
import cv2
import numpy as np
from sklearn.metrics import accuracy_score
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model

# ✅ Load your .h5 model
model_path = "/kaggle/input/violence-detection/tensorflow2/default/1/final_violence_model1.h5"
model = load_model(model_path)

# ✅ Parameters
img_size = (224, 224)
test_dir = "/kaggle/input/smartcity-cctv-violence-detection-dataset-scvd/SCVD/SCVD_converted/Test"  # Change to your test path
class_labels = ['Normal', 'Violence', 'Weaponized']  # Adjust based on your folders

def preprocess_frame(frame):
    frame = cv2.resize(frame, img_size)
    frame = frame.astype("float32") / 255.0
    frame = img_to_array(frame)
    return frame

y_true = []
y_pred = []

for class_name in os.listdir(test_dir):
    class_path = os.path.join(test_dir, class_name)
    for video_file in os.listdir(class_path):
        if not video_file.endswith('.avi'):
            continue
        video_path = os.path.join(class_path, video_file)
        cap = cv2.VideoCapture(video_path)

        frames = []
        count = 0
        while cap.isOpened() and count < 20:
            ret, frame = cap.read()
            if not ret:
                break
            frames.append(preprocess_frame(frame))
            count += 1
        cap.release()

        if not frames:
            continue

        frames = np.array(frames)
        preds = model.predict(frames)
        avg_pred = np.mean(preds, axis=0)
        pred_class = np.argmax(avg_pred)

        y_true.append(class_labels.index(class_name))
        y_pred.append(pred_class)

# ✅ Accuracy
acc = accuracy_score(y_true, y_pred)
print(f"✅ Average accuracy on test videos: {acc*100:.2f}%")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 158ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━

In [1]:
import os
import cv2
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array

# === Load the model ===
model_path = "/kaggle/input/violence-detection/tensorflow2/default/1/final_violence_model1.h5"
model = load_model(model_path)

# === Parameters ===
img_size = (224, 224)
test_dir = "/kaggle/input/smartcity-cctv-violence-detection-dataset-scvd/SCVD/SCVD_converted/Test"  # ✅ Change this to your real path
class_labels = ['Normal', 'Violence', 'Weaponized']   # ✅ Make sure this matches your folders

def preprocess_frame(frame):
    frame = cv2.resize(frame, img_size)
    frame = frame.astype("float32") / 255.0
    frame = img_to_array(frame)
    return frame

# === Accuracy counters ===
class_correct = {label: 0 for label in class_labels}
class_total = {label: 0 for label in class_labels}

# === Loop through each class ===
for class_name in class_labels:
    class_path = os.path.join(test_dir, class_name)
    
    for video_file in os.listdir(class_path):
        if not video_file.endswith(".avi"):
            continue
        
        video_path = os.path.join(class_path, video_file)
        cap = cv2.VideoCapture(video_path)

        frames = []
        count = 0
        while cap.isOpened() and count < 20:
            ret, frame = cap.read()
            if not ret:
                break
            frames.append(preprocess_frame(frame))
            count += 1
        cap.release()

        if not frames:
            continue

        frames = np.array(frames)
        preds = model.predict(frames)
        avg_pred = np.mean(preds, axis=0)
        pred_class_idx = np.argmax(avg_pred)

        true_idx = class_labels.index(class_name)

        # === Accuracy counting ===
        class_total[class_name] += 1
        if pred_class_idx == true_idx:
            class_correct[class_name] += 1

# === Print accuracy per class ===
print("\n✅ Accuracy per class:")
for label in class_labels:
    correct = class_correct[label]
    total = class_total[label]
    acc = (correct / total) * 100 if total > 0 else 0
    print(f"{label}: {acc:.2f}% ({correct}/{total})")


1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━